In [1]:
import time
import re
import codecs
import urllib2 as ul
import xml.etree.ElementTree as ET

def ReadSent(pageno,src='../han/'):
    dat=codecs.open(src+str(pageno)+'.d',encoding='utf-8')
    dat = dat.read()
    dat = Clean(dat)
    dat = dat.replace('?','.')
    dat = dat.replace('!','.')
    return [' '.join(i.split()) for i in dat.split('.')]
    
def Clean(sentence,comma=0):
    if comma==0:
        for i in ['\n','_','-','(',')','"','\'',u'▲','...','[',']',u'■','<','>','\r',u'▶',u'◀']:
            sentence=sentence.replace(i,' ')
    else:
        for i in ['\n','_','-','(',')','"','\'',u'▲','...','[',']',u'■','<','>',',','\r']:
            sentence=sentence.replace(i,' ')
    return sentence.strip()

def LoadDB():
    sent1=[]
    f = open('../lists/han.list')
    for line in f:
        tmp=ReadSent(int(line.split('.')[0]))
        for j in tmp:
            sent1.append(j)

    sent2=[]
    f = open('../lists/dat.list')
    for line in f:
        tmp=ReadSent(int(line.split('.')[0]),src='../dat/')
        for j in tmp:
            sent2.append(j)
        
    sent3=[]
    f = open('../lists/oro.list')
    for line in f:
        tmp=ReadSent(int(line.split('.')[0]),src='../oro/')
        for j in tmp:
            sent3.append(j)

    return sent1+sent2+sent3

def ShortSent(sent):
    return sorted(sent, key = lambda x: len(x))

def SearchDB(sent,token):
    matches = [i for i in sent if i.find(token)>0]
    return sorted(matches, key=lambda x: len(x))

def SearchDBList(sent,tokens):
    matches=[]
    for token in tokens:
        matches += [i for i in sent if i.find(token)>0]
    return sorted(matches, key=lambda x: len(x))

def Simplify(sent):
    return list(set(sent))

#a la https://github.com/mouuff/Google-Translate-API/blob/master/python/GoogleTranslate.py
def translate(to_translate, to_langage="auto", langage="auto"):
    to_translate=to_translate.encode('utf-8')
    agents = {'User-Agent':"Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1;SV1; .NET CLR 1.1.4322; .NET CLR 2.0.50727; .NET CLR 3.0.04506.30)"}
    before_trans = 'class="t0">'
    link = "http://translate.google.com/m?hl=%s&sl=%s&q=%s"% (to_langage, langage, to_translate.replace(" ", "+"))
    request = ul.Request(link, headers=agents)
    page = ul.urlopen(request).read()
    result = page[page.find(before_trans)+len(before_trans):]
    result = result.split("<")[0]
    return result

def LoadWordBank(wbank,path='./xml/'):
    tree = ET.parse(path+wbank)
    root = tree.getroot()
    mywords={}
    for j,k in zip(root.iter('Word'),root.iter('English')):
        mywords[j.text]=k.text
    return mywords

def WCount(sent,write=0,n=2000,name='freq.wlist'):
    mywords={}
    for i in sent:
        i=i.replace(',','')
        for j in i.split():
            try:
                mywords[j]+=1
            except KeyError:
                mywords[j]=1
    freq = sorted(mywords.keys(), key=lambda x: mywords[x])
    freq.reverse()
    if write==1:
        dat=codecs.open('./output/'+name,'w',encoding='utf-8')
        for i in freq[:n]:
            try:
                dat.write(i)
                dat.write(', '+translate(i))
                dat.write('\n')
            except UnicodeDecodeError:
                continue
        
    return mywords, freq
        
def LoadFreqList(name,src='./output/'):
    f = open(src+name)
    keys=[]
    vals=[]
    for line in f:
        tmp = line.decode('utf-8').split(',')
        keys.append(tmp[0])
        vals.append(int(tmp[1]))
    return keys,vals

def MergeFreqList(name1,name2):
    dic1={}
    keys,vals=LoadFreqList(name1)
    keys1,vals1=LoadFreqList(name2)
    for i,j in zip(keys,vals):
        dic1[i]=j
    for i,j in zip(keys1,vals1):
        try:
            dic1[i]+=j
        except KeyError:
            dic1[i]=j
    keyt=sorted(dic1.keys(), key=lambda x: dic1[x])
    keyt.reverse()
    valt=[dic1[i] for i in keyt]
    return keyt,valt

In [2]:
sent = LoadDB()

##KoreanClass101 Lower Beginner #15 'Making Plans in Korea'

In [3]:
# 만날까요?  mannalkkayo? Shall we meet? expression
# 있다
# ittda to exist verb
# 남산
# namsan Namsan Mountain proper noun
# 알겠다 algettda to know, to verb understand
# 약속 yakssok appointment, noun promise
# 없다
# eoptda to not exist verb
# 괜찮다 gwaenchanhta to be okay, to be all adjective right
# 보다 boda to see, to look at, to verb watch
# 역
# yeok station noun
# Myeongdong, a
# 명동 myeongdong shopping district in proper noun downtown Seoul

tags = [u'있다 ',u'남산',u'알겠다 ',u' 약속',u'없다 ',u' 괜찮다 ',u' 보다 ',u'  역',u'명동']
for tag in tags:
    stag = SearchDB(sent,tag)
    stag=Simplify(stag)
    stag=ShortSent(stag)
    print tag, len(stag)
    for i in range(min([10,len(stag)])):print stag[i]
    print ''
    print tag
    for i in range(min([30,len(stag)])):print stag[i],translate(stag[i])
    print ''

있다  2360
신은 살아있다 5
그런 게 있다 싶다
자신 있다 고 말했다
○● 효험이 있다 2
다뉴브 강가에 있다 7
야맛있다 9단 감사합니다
조금 있다 보기로 하고…
자신있다 고 말한 바 있다
뒤에 벤이 웃고 있다 15
민상연이 착수하고 있다 음

있다 
신은 살아있다 5 God is alive 5
그런 게 있다 싶다 I want it like that.
자신 있다 고 말했다 Himself has said,
○● 효험이 있다 2 ○ ● There are two efficacy
다뉴브 강가에 있다 7 In the Danube river 7
야맛있다 9단 감사합니다 9&#39;re delicious, thank you
조금 있다 보기로 하고… There is a little more ...
자신있다 고 말한 바 있다 There are bar and told his
뒤에 벤이 웃고 있다 15 Ben 15 is back laughing
민상연이 착수하고 있다 음 Minsang kite launching Well
자신감은 늘 넘쳐 있다 웃음 Confidence is always full of laughter
너무 재미있다 라고 말했다고 Told that too funny
긴장되는 것보다 재미있다 고 And fun than being nervous
8%가 있다 고 대답해 45 It has 8% and 45 answers
잘 준비하고 있다 고 말했다 Well prepared and have said,
그나마 있다 해도 보수가 작다 Dont even if there is less maintenance
한국이 우승할 수 있다 보는가 This look can win South Korea
7%가 있다 라고 응답을 했다 The response was that there is a 7%
자신 있다 고 소감을 피력했다 There was myself and express a comment
shantou1 야맛있다 연기 shantou1 be postponed delicious
물론 부러